In [1]:
import pandas as pd
import numpy as np

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash

import datetime

In [2]:
# load data
df = pd.read_csv('../data/policejune.csv')
df = df.dropna(subset = ['Object of search'])

# ethnicity data prep
df['Stop and Search'] = 1
conditions = [
    (df['Officer-defined ethnicity'] == 'Black'),
    (df['Officer-defined ethnicity'] == 'White'),
    (df['Officer-defined ethnicity'] == 'Asian'),
    (df['Officer-defined ethnicity'] == 'Other')]
choices = ['Black', 'White', 'Asian', 'Other']
df['Officer-defined ethnicity new'] = np.select(conditions, choices, default='Not stated')
conditions = [
    (df['Self-defined ethnicity'].str.startswith('Black') == 1),
    (df['Self-defined ethnicity'].str.startswith('White') == 1),
    (df['Self-defined ethnicity'].str.startswith('Asian') == 1),
    (df['Self-defined ethnicity'].str.startswith('Other') == 1),
    (df['Self-defined ethnicity'].str.startswith('Mixed') == 1)]
choices = ['Black', 'White', 'Asian', 'Other', 'Mixed']
df['Self-defined ethnicity new'] = np.select(conditions, choices, default='Not stated')
selfDefinedEthnicities = df['Self-defined ethnicity new'].unique()

conditions = [
    (df['Self-defined ethnicity new'] == df['Officer-defined ethnicity new'])]
choices = ['Match']
df['Ethnicity matches'] = np.select(conditions, choices, default='Mismatch')

df_match = df[df['Ethnicity matches'] == 'Match']
df_mismatch = df[df['Ethnicity matches'] == 'Mismatch']

In [3]:
'''
dash layout
'''

app = JupyterDash(__name__)

app.layout = html.Div(style={'display': 'flex', 'flexDirection': 'column', 'height':'90vh', 'backgroundColor': 'lightgrey', 'margin': 0}, children=[
    html.Div(style={'display': 'flex', 'flexDirection': 'row', 'margin': 0, 'height': '6%'}, children=[
        html.H4("police stop and search data in London, 06/2021", style={'alignSelf':'center', 'marginLeft': '1rem'}),
        html.P("select an area of interest on the map below to get started", style={'alignSelf':'center', 'marginLeft': '1rem'})
    ]),
    html.Div(style={'display': 'flex', 'flexDirection': 'row', 'height': '47%'}, children=[
        html.Div(style = {'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center', 'width': '60%'}, children=[
            dcc.Graph(id='geo', style={'height': '90%'}),
            dcc.Dropdown(
                id='dropdown',
                options=[{'label': x, 'value': x} for x in df['Outcome'].unique()],
                value='Arrest',
                clearable=True,
                multi=True,
                style={'width': '90%', 'margin': 'auto'}
            ),
        ]),
        html.Div(style = {'width': '40%'}, children=[
            dcc.Graph(id='ethnicitypie', style={'cursor': 'pointer', 'height': '90%'}),
            dcc.Dropdown(
                id='ethnicity', 
                value='Black', 
                options=[{'value': x, 'label': x} for x in selfDefinedEthnicities],
                clearable=True,
                style={'width': '90%', 'margin': 'auto'}
            ),
        ]),
    ]),
    html.Div(style={'display': 'flex', 'flexDirection': 'row', 'height': '47%'}, children=[
        dcc.Graph(style={'width': '30%', 'height': '100%'}, id='stackedbar'),
        dcc.Graph(style={'width': '30%', 'height': '100%'}, id='bar'),
        html.Div(style={'display': 'flex', 'flexDirection': 'column', 'width': '40%', 'height': '100%', 'margin': 0}, children=[
            html.Div(id='mismatchpie', style={'height': '100%', 'margin': 0})
        ])
    ])
])

'''
helper functions
'''

outcome_colormap={
    'Arrest': 'rgb(228,26,28)',
    'Community resolution': 'rgb(55,126,184)',
    'A no further action disposal': 'rgb(77,175,74)',
    'Summons / charged by post': 'rgb(152,78,163)',
    'Penalty Notice for Disorder': 'rgb(255,127,0)',
    'Caution (simple or conditional)': 'rgb(166,86,40)',
}

xlab_dict = {'weekday' : [
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday'
]}

def get_selected_rows(dff, outcome, selected_data):
    if type(outcome) == str:
        outcome = [outcome]
    dff = df[df['Outcome'].isin(outcome)]
    
    if selected_data and selected_data['points']:
        selected_indices = pd.Index([])
        for point in selected_data['points']:
            temp = dff[dff['Latitude'] == point.get('lat')]
            temp = temp[temp['Longitude'] == point.get('lon')]
            selected_indices = selected_indices.union(temp.index)
        dff = dff[dff.index.isin(selected_indices)]
    
    return dff

@app.callback(
    Output('geo', 'selectedData'),
    [Input(component_id='dropdown', component_property='value')]
)
def clear_selected(new_dropdown_value):
    return None

def weekday_compute(dff):
    # stacked bar chart data prep
    df_sub = dff[['Date','Outcome']]
    
    #Convert datetime column/series to day of the week
    df_sub['weekday'] = pd.to_datetime(df_sub['Date'], errors='coerce')
    df_sub['weekday'] = df_sub['weekday'].dt.day_name()
    df_weekOutcome = df_sub.groupby(['Outcome', 'weekday']).size().to_frame('size')
    df_weekOutcome = df_weekOutcome.reset_index()
    return df_weekOutcome

def parse_pie_click_label(clickData):
    if clickData and clickData.get('points'):
        return clickData.get('points')[0].get('label')
    return None

'''
chart callback funtions
'''

@app.callback(
    Output('geo', 'figure'),
    [Input('dropdown', 'value')])
def update_geo(outcome):
    if type(outcome) == str:
        outcome = [outcome]
    dff = df[df['Outcome'].isin(outcome)]
    
    fig = px.scatter_mapbox(dff, lat='Latitude', lon='Longitude', color='Outcome', color_discrete_map=outcome_colormap)
    fig.update_layout(mapbox_style='carto-positron')
    fig.update_layout(showlegend=False, margin = dict(l = 1, r = 1, t = 1, b = 1))
    fig.update_layout(paper_bgcolor="lightgrey")
    return fig


@app.callback(
    Output('bar', 'figure'),
    [Input('dropdown', 'value'),
     Input('geo','selectedData')])
def update_bar(outcome, selected_data):
    dff = get_selected_rows(df, outcome, selected_data)
    dff = dff.groupby('Object of search').count().reset_index()
    fig = px.bar(dff, x='Object of search', y='Stop and Search', color='Object of search', title='objects the police searched for',
                 labels={
                     'Object of search': '',  # hide the label
                     'Stop and Search': '# of searches for an object'
                 },
                 color_discrete_map=outcome_colormap)
    
    fig.update_traces(marker_line_width = 0, selector=dict(type='bar'))
    fig.update_layout(showlegend=False)
    fig.update_layout(paper_bgcolor="lightgrey")
    return fig


@app.callback(
    Output('stackedbar', 'figure'),
    [Input('dropdown', 'value'),
     Input('geo','selectedData')])
def update_stackedbar(outcome, selected_data):
    dff = get_selected_rows(df, outcome, selected_data)
    df_weekOutcome = weekday_compute(dff)
    fig = px.bar(df_weekOutcome, x='weekday', y='size', color='Outcome', title='police searches per weekday', category_orders=xlab_dict,
            labels={
                'weekday': '', # hide the label
                'size': '# of searches per weekday'
            },
            color_discrete_map=outcome_colormap)
    
    fig.update_layout(showlegend=False)
    fig.update_layout(paper_bgcolor="lightgrey")
    return fig


@app.callback(
    Output('ethnicitypie', 'figure'), 
    [
        Input('dropdown', 'value')
    ])
def update_pie_ethnicity(ignored_value):
    fig = px.pie(df, values='Stop and Search', names='Ethnicity matches', labels={'Stop and Search':'Number of Stop and Searches'}, title="self-defined and officer-defined ethnicity")
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=False)
    fig.update_layout(paper_bgcolor="lightgrey")
    return fig


@app.callback(
    Output('mismatchpie', 'children'), 
    [Input('ethnicitypie', 'clickData'),
     Input('ethnicity', 'value')])
def update_pie_match_mismatch(click_data, eth_value):
    label = parse_pie_click_label(click_data)
    
    if label:
        if label == 'Match':
            label = 'outcomes of police searching %s people' % eth_value
            dd_label = '(choose ethnicity with dropdown or clear to view all)'
            if eth_value:
                dff = df_match[(df_match['Self-defined ethnicity new'] == eth_value)]
            else:
                dff = df_match
            fig = px.pie(dff, values='Stop and Search', names='Outcome', labels={'Stop and Search':'Number of Stop and Searches'}, color='Outcome', color_discrete_map=outcome_colormap)
        elif label == 'Mismatch':
            label = 'Mismatch'
            dd_label = '(dropdown above does nothing)'
            dff = df_mismatch
            fig = px.pie(dff, values='Stop and Search', names='Outcome', labels={'Stop and Search':'Number of Stop and Searches'}, color='Outcome', color_discrete_map=outcome_colormap)
        
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.update_layout(showlegend=False)
        fig.update_layout(paper_bgcolor="lightgrey")
        res = html.Div(style={'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center', 'textAlign': 'center', 'height': '100%'}, children=[
            html.H3(label, style={'margin': 0, 'height': '10%'}),
            html.P(dd_label, style={'margin': 0, 'height': '10%'}),
            dcc.Graph(figure=fig, style={'margin': 0, 'height': '90%'})
        ])
        return res
    
    res = html.Div(style={'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center', 'textAlign': 'center'}, children=[
        html.H3('please click on "Match" or "Mismatch" in the pie chart above'),
    ])
    return res


app.run_server(mode='inline', port=8050, threaded=True)